In [2]:
import pandas as pd
import os

In [3]:
os.chdir("/Users/deeptimurthy/BENG203")
wd = '/Users/deeptimurthy/BENG203'

In [ ]:

cancer_df.columns = [f'C{i+1}' for i in range(cancer_df.shape[1])]
normal_df.set_index(normal_df.columns[0], inplace=True)
normal_df.columns = [f'N{i+1}' for i in range(normal_df.shape[1])]

In [63]:
combined_df = pd.concat([cancer_df, normal_df], axis=1, join='inner')
combined_df.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,N87,N88,N89,N90,N91,N92,N93,N94,N95,N96
ENSG00000000003,26.432148,17.260895,0.000000,69.795165,19.331552,0.000000,2.928341,53.467303,14.997727,25.816814,...,14.134044,10.803910,6.068982,12.714350,15.983392,0.000000,38.189128,15.608352,51.519267,30.805593
ENSG00000000005,24.817762,5.402219,7.557640,0.000000,0.000000,14.330953,0.000000,16.733900,14.081717,0.000000,...,0.000000,12.172853,136.759411,28.650721,0.000000,0.000000,5.976111,6.280734,241.863225,0.000000
ENSG00000000419,66.208114,0.000000,10.081027,0.000000,18.158365,9.557926,0.000000,0.000000,0.000000,0.000000,...,79.657691,8.118597,0.000000,0.000000,0.000000,179.178412,15.942897,0.000000,0.000000,0.000000
ENSG00000000457,8.127159,6.318156,15.910242,15.328639,19.105460,6.704296,34.729107,0.000000,2.195899,20.102628,...,18.624986,15.660412,33.988677,0.000000,36.858404,83.788404,18.172309,2.938248,14.547636,10.148436
ENSG00000000460,8.035515,20.406574,0.000000,17.681754,11.019179,1.933370,0.000000,18.060385,10.131976,23.188602,...,17.903438,19.706680,23.062537,1.932615,28.344382,3.020339,27.411791,1.694651,3.729079,7.804221


In [4]:
oncogenes_df = pd.read_csv("C:/Users/noapo/Downloads/cancerGeneList.tsv", sep="\t")
oncogenes_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/noapo/Downloads/cancerGeneList.tsv'

In [65]:
from pybiomart import Dataset

dataset = Dataset(name='hsapiens_gene_ensembl', host='http://www.ensembl.org')
mapping_df = dataset.query(attributes=['ensembl_gene_id', 'hgnc_symbol'])
mapping_df.columns = ['Gene stable ID', 'HGNC symbol']
mapping_df = mapping_df[mapping_df['HGNC symbol'] != '']

ensg_to_symbol = dict(zip(mapping_df['Gene stable ID'], mapping_df['HGNC symbol']))

combined_df['gene_symbol'] = combined_df.index.map(ensg_to_symbol)
combined_df = combined_df[combined_df['gene_symbol'].notnull()]
combined_df.set_index('gene_symbol', inplace=True)
combined_df.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,N87,N88,N89,N90,N91,N92,N93,N94,N95,N96
gene_symbol,,,,,,,,,,,,,,,,,,,,,
TSPAN6,26.432148,17.260895,0.000000,69.795165,19.331552,0.000000,2.928341,53.467303,14.997727,25.816814,...,14.134044,10.803910,6.068982,12.714350,15.983392,0.000000,38.189128,15.608352,51.519267,30.805593
TNMD,24.817762,5.402219,7.557640,0.000000,0.000000,14.330953,0.000000,16.733900,14.081717,0.000000,...,0.000000,12.172853,136.759411,28.650721,0.000000,0.000000,5.976111,6.280734,241.863225,0.000000
DPM1,66.208114,0.000000,10.081027,0.000000,18.158365,9.557926,0.000000,0.000000,0.000000,0.000000,...,79.657691,8.118597,0.000000,0.000000,0.000000,179.178412,15.942897,0.000000,0.000000,0.000000
SCYL3,8.127159,6.318156,15.910242,15.328639,19.105460,6.704296,34.729107,0.000000,2.195899,20.102628,...,18.624986,15.660412,33.988677,0.000000,36.858404,83.788404,18.172309,2.938248,14.547636,10.148436
FIRRM,8.035515,20.406574,0.000000,17.681754,11.019179,1.933370,0.000000,18.060385,10.131976,23.188602,...,17.903438,19.706680,23.062537,1.932615,28.344382,3.020339,27.411791,1.694651,3.729079,7.804221


In [ ]:
oncogene_genes = oncogenes_df['Hugo Symbol'].tolist()

# save oncogene genes
oncogenes_df = pd.read_csv('oncogene_genes.csv')

filtered_df = combined_df.loc[combined_df.index.intersection(oncogene_genes)]
filtered_df.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,N87,N88,N89,N90,N91,N92,N93,N94,N95,N96
gene_symbol,,,,,,,,,,,,,,,,,,,,,
RAD52,3.814472,4.151586,17.424057,3.147584,41.846581,5.506643,0.000000,9.644957,5.410874,7.620693,...,33.145289,2.338699,3.284347,0.000000,2.883241,17.205119,2.296310,26.546970,7.965901,11.114031
LASP1,13.581055,13.549541,0.000000,0.000000,29.487735,13.070576,3.761514,38.155473,5.351358,1.507374,...,12.103647,2.775569,0.000000,4.899553,22.241913,45.942857,14.988943,4.296268,0.000000,3.297535
RECQL,10.576124,2.302163,0.000000,3.490836,11.602513,9.160733,35.150939,10.696763,0.000000,5.634499,...,5.655363,5.187478,0.000000,45.785793,3.197666,0.000000,5.093456,26.765437,0.000000,9.244532
UPF1,13.938348,15.170186,26.882348,10.734726,34.404824,18.780213,4.632567,10.964596,14.059884,11.138618,...,17.390900,22.788714,0.000000,2.681841,4.214226,0.000000,5.593918,2.351624,5.174748,5.414859
HOXA11,6.927888,15.080318,0.000000,0.000000,4.750140,0.000000,5.756404,11.678188,29.481877,0.000000,...,9.261362,8.495142,0.000000,0.000000,0.000000,0.000000,8.341169,0.000000,19.290348,0.000000
